## Recommender System - Collaborative Filtering

In [55]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [56]:
data=pd.read_table('/Users/nag/Code Files/Datasets/u.data',sep='\t',header=None)
movie=pd.read_table('/Users/nag/Code Files/Datasets/u.item',sep='|',encoding='latin-1',header=None)

In [57]:
data.sample(5)
movie.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
611,612,Lost Horizon (1937),01-Jan-1937,NaN,http://us.imdb.com/M/title-exact?Lost%20Horizo...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1098,1099,"Red Firecracker, Green Firecracker (1994)",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pao%20Da%20Sh...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,942,What's Love Got to Do with It (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?What's%20Love...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1096,1097,"Hate (Haine, La) (1995)",09-Feb-1996,NaN,"http://us.imdb.com/M/title-exact?Haine,%20La%2...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
464,465,"Jungle Book, The (1994)",01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Jungle%20Book...,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0


In [58]:
data.columns=['userId','itemId','rating','timestamp']

In [59]:
movie.columns=['itemId','title','release date','video','url','genre1','genre2','genre3','genre4','genre5','genre6','genre7','genre8','genre9','genre10','genre11','genre12','genre13','genre14','genre15','genre16','genre17','genre18','genre19']

In [60]:
movie.head()
data.head()
movie.shape
data.shape
#data[data.userId==196]

(100000, 4)

In [61]:
Mean=data.groupby(by='userId',as_index=False)['rating'].mean()
rating_avg=pd.merge(data,Mean,on='userId')
Mean

,userId,rating
0,1,3.610294
1,2,3.709677
2,3,2.796296
3,4,4.333333
4,5,2.874286
...,...,...
938,939,4.265306
939,940,3.457944
940,941,4.045455
941,942,4.265823


In [62]:
rating_avg.head()
rating_avg.shape

(100000, 5)

In [63]:
rating_avg['stdzd_rating']=rating_avg['rating_x']-rating_avg['rating_y']

In [64]:
rating_avg.sample(10)

,userId,itemId,rating_x,timestamp,rating_y,stdzd_rating
87380,829,319,4,892312728,3.546875,0.453125
66615,606,157,4,880926018,3.966102,0.033898
93281,879,751,2,887760879,3.689655,-1.689655
15469,198,692,2,884208377,3.276243,-1.276243
63697,567,100,1,882425791,3.954839,-2.954839
63379,566,153,2,881649747,3.442953,-1.442953
5530,63,285,3,875747470,3.118280,-0.118280
34976,327,1170,4,887819860,3.313380,0.686620
6034,290,54,3,880475218,3.353333,-0.353333
48206,424,310,3,880858829,3.363636,-0.363636


In [65]:
# Trick/pre-requisite for recommender system
# dataframe should be in the format of real rating matrix 
# this is done by converting dataframe to a pivot table and then if required to csr/sparse matrix


In [66]:
check=pd.pivot_table(rating_avg,index='userId',columns='itemId',values='stdzd_rating')

In [67]:
check.head()

itemId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,1.389706,-0.610294,0.389706,-0.610294,-0.610294,1.389706,0.389706,-2.610294,1.389706,-0.610294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.290323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.709677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.125714,0.125714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
final_movie=check.fillna(check.mean(axis=0))
final_movie.head()

itemId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,1.389706,-0.610294,0.389706,-0.610294,-0.610294,1.389706,0.389706,-2.610294,1.389706,-0.610294,...,-1.147059,-0.137056,-0.45933,-1.45933,-0.211982,-2.121495,-0.121495,-1.121495,0.019337,-0.365931
2,0.290323,-0.253455,-0.406476,-0.029170,-0.206708,0.099592,0.241369,0.370904,0.316282,-1.709677,...,-1.147059,-0.137056,-0.45933,-1.45933,-0.211982,-2.121495,-0.121495,-1.121495,0.019337,-0.365931
3,0.299264,-0.253455,-0.406476,-0.029170,-0.206708,0.099592,0.241369,0.370904,0.316282,0.251461,...,-1.147059,-0.137056,-0.45933,-1.45933,-0.211982,-2.121495,-0.121495,-1.121495,0.019337,-0.365931
4,0.299264,-0.253455,-0.406476,-0.029170,-0.206708,0.099592,0.241369,0.370904,0.316282,0.251461,...,-1.147059,-0.137056,-0.45933,-1.45933,-0.211982,-2.121495,-0.121495,-1.121495,0.019337,-0.365931
5,1.125714,0.125714,-0.406476,-0.029170,-0.206708,0.099592,0.241369,0.370904,0.316282,0.251461,...,-1.147059,-0.137056,-0.45933,-1.45933,-0.211982,-2.121495,-0.121495,-1.121495,0.019337,-0.365931


In [69]:
final_user=check.apply(lambda x: x.fillna(x.mean()),axis=1)
final_user.head()

itemId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,1.389706e+00,-6.102941e-01,3.897059e-01,-6.102941e-01,-6.102941e-01,1.389706e+00,3.897059e-01,-2.610294e+00,1.389706e+00,-6.102941e-01,...,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16,2.579636e-16
2,2.903226e-01,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,-1.709677e+00,...,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16,4.655774e-16
3,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,...,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16,1.151342e-16
4,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,...,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16,2.960595e-16
5,1.125714e+00,1.257143e-01,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,...,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16,2.131628e-16


In [70]:
b=cosine_similarity(final_user)
np.fill_diagonal(b,0)
similarity_with_user=pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head(5)

userId,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.043411,0.011051,0.059303,0.134514,0.103373,0.110556,0.180891,0.012253,-0.000621,...,0.025835,-0.047952,0.087224,0.007718,0.074378,0.078714,0.067433,0.028790,-0.031270,0.032123
2,0.043411,0.000000,0.013658,-0.017016,0.035770,0.094503,0.089408,0.055640,0.027294,0.097846,...,0.012853,-0.028798,0.056659,0.197835,0.090009,0.032505,0.015053,-0.017344,0.012068,0.039173
3,0.011051,0.013658,0.000000,-0.059638,0.016037,-0.017158,0.016141,0.041177,-0.010093,0.023856,...,0.001615,0.000658,-0.006888,0.036157,-0.018513,-0.006240,-0.023907,0.034414,-0.009187,0.001489
4,0.059303,-0.017016,-0.059638,0.000000,0.007373,-0.053929,-0.025604,0.136046,0.016082,-0.013588,...,0.011895,0.002174,-0.028000,-0.025021,0.022882,-0.005960,0.279818,0.258594,0.064504,-0.019222
5,0.134514,0.035770,0.016037,0.007373,0.000000,0.038484,0.067874,0.140106,0.010195,0.014335,...,0.070014,-0.070821,0.024278,0.038672,0.093567,0.051782,0.029540,0.036234,0.043318,0.099324


### Recommend movies to users

In [71]:
movie

,itemId,title,release date,video,url,genre1,genre2,genre3,genre4,genre5,...,genre10,genre11,genre12,genre13,genre14,genre15,genre16,genre17,genre18,genre19
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
def recommend_movies_to_user(user1,user2):
    common_movies=rating_avg[rating_avg.userId==user1].merge(rating_avg[rating_avg.userId==user2],on='itemId',how='right')
    
    return common_movies.merge(movie, on ='itemId')

In [73]:
common_movies=rating_avg[rating_avg.userId==736].merge(rating_avg[rating_avg.userId==145],on='rating_x',how='left')
#common_movies.columns=['userid','itemid','rating_first_user','timestamp','avg_rating_first_user','userid2','itemid2','rating_first_user2','timestamp2','avg_rating_first_user2','other']
#common_movies

In [74]:
rating_avg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   userId        100000 non-null  int64  
 1   itemId        100000 non-null  int64  
 2   rating_x      100000 non-null  int64  
 3   timestamp     100000 non-null  int64  
 4   rating_y      100000 non-null  float64
 5   stdzd_rating  100000 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 5.3 MB


In [80]:
recommendation=recommend_movies_to_user(736,145)
recommendation_result=recommendation.loc[:,['rating_x_x','rating_x_y','title']]
recommendation_result[recommendation_result.rating_x_x.isnull()]

,rating_x_x,rating_x_y,title
10,NaN,2,Sense and Sensibility (1995)
11,NaN,5,Jerry Maguire (1996)
12,NaN,2,Mr. Holland's Opus (1995)
13,NaN,5,Pulp Fiction (1994)
14,NaN,1,Eye for an Eye (1996)
...,...,...,...
311,NaN,1,Three Colors: Red (1994)
312,NaN,5,Dances with Wolves (1990)
313,NaN,2,Independence Day (ID4) (1996)
314,NaN,2,Jane Eyre (1996)
